# Basic Hypergeometric Summation

A **basic hypergeometric series** (or $q$-hypergeometric series) is defined as:

$${}_r\phi_s\left(\begin{matrix} a_1, a_2, \ldots, a_r \\ b_1, b_2, \ldots, b_s \end{matrix}; q, z\right) = \sum_{n=0}^{\infty} \frac{(a_1;q)_n (a_2;q)_n \cdots (a_r;q)_n}{(b_1;q)_n (b_2;q)_n \cdots (b_s;q)_n} \frac{z^n}{(q;q)_n}$$

where $(a;q)_n = \prod_{k=0}^{n-1}(1 - aq^k)$ is the $q$-Pochhammer symbol.

q-Kangaroo can evaluate these series, apply summation formulas, and perform
classical transformations. Parameters are specified as tuples `(num, den, power)`
representing the $q$-monomial $\frac{\text{num}}{\text{den}} \cdot q^{\text{power}}$.

In [ ]:
from q_kangaroo import QSession, phi

s = QSession()

# Evaluate 2phi1(q^2, q^3; q^5; q, q)
# upper = [q^2, q^3], lower = [q^5], z = q
result = phi(s, [(1,1,2), (1,1,3)], [(1,1,5)], 1, 1, 1, 20)
result

1 + q + 2*q^2 + 2*q^3 + 2*q^4 + q^5 + 2*q^6 + q^7 + q^8 + q^9 + q^10 + q^11 + q^12 + q^14 + q^15 + q^16 + q^19 + O(q^20)

## The $q$-Gauss Summation Formula

The **$q$-Gauss formula** (DLMF 17.7.2) provides a closed form for certain ${}_2\phi_1$ series:

$${}_2\phi_1\left(\begin{matrix} a, b \\ c \end{matrix}; q, \frac{c}{ab}\right) = \frac{(c/a;q)_\infty\,(c/b;q)_\infty}{(c;q)_\infty\,(c/(ab);q)_\infty}$$

The `try_summation` function automatically detects when a series matches a known
summation formula and returns the closed form:

In [ ]:
from q_kangaroo import try_summation

# 2phi1(q^2, q^3; q^8; q, q^3)  [z = c/(ab) = q^8/(q^2*q^3) = q^3]
closed = try_summation(s, [(1,1,2), (1,1,3)], [(1,1,8)], 1, 1, 3, 20)
if closed is not None:
    print("Closed form found!")
closed

Closed form found!


1 + q^3 + q^4 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)

## Comparing Direct Evaluation vs Closed Form

We can verify the summation formula by computing the series both ways
and confirming they match:

In [ ]:
# Direct evaluation via phi()
direct = phi(s, [(1,1,2), (1,1,3)], [(1,1,8)], 1, 1, 3, 20)
print("Direct: ", direct)
print("Closed: ", closed)
print("Match:", repr(direct) == repr(closed))

Direct:  1 + q^3 + q^4 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)
Closed:  1 + q^3 + q^4 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)
Match: True


## Heine's First Transformation

**Heine's first transformation** (DLMF 17.6.1) rewrites a ${}_2\phi_1$ as a
product prefactor times a different ${}_2\phi_1$:

$${}_2\phi_1(a, b; c; q, z) = \frac{(b;q)_\infty (az;q)_\infty}{(c;q)_\infty (z;q)_\infty} \cdot {}_2\phi_1(c/b, z; az; q, b)$$

The `heine1` function returns both the prefactor and the transformed result:

In [ ]:
from q_kangaroo import heine1

# Apply Heine's first transformation to the same 2phi1
prefactor, transformed = heine1(s, [(1,1,2), (1,1,3)], [(1,1,8)], 1, 1, 3, 20)
print("Prefactor:  ", prefactor)
print("Transformed:", transformed)

Prefactor:   1 - q^5 - q^6 - q^7 + q^11 + q^12 + q^13 - q^18 + O(q^20)
Transformed: 1 + q^3 + q^4 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)


The transformed result equals the original series, confirming the identity.
The prefactor is an infinite product involving $q$-Pochhammer symbols.

## $q$-Vandermonde Summation

The **$q$-Chu-Vandermonde formula** (DLMF 17.6.2) provides closed forms for
**terminating** ${}_2\phi_1$ series -- those with an upper parameter $q^{-n}$
that makes the sum finite.

**First form** ($z = cq^n/a$):

$${}_2\phi_1\left(\begin{matrix} a, q^{-n} \\ c \end{matrix}; q, \frac{cq^n}{a}\right) = \frac{(c/a;q)_n}{(c;q)_n}$$

**Second form** ($z = q$):

$${}_2\phi_1\left(\begin{matrix} a, q^{-n} \\ c \end{matrix}; q, q\right) = a^n \cdot \frac{(c/a;q)_n}{(c;q)_n}$$

Since $q^{-n}$ makes the series terminate after $n+1$ terms, these are polynomial identities.

In [ ]:
# q-Vandermonde (second form): 2phi1(q^{-3}, q^2; q^5; q, q)
# Upper = [q^{-3}, q^2], lower = [q^5], z = q
# This terminates after 4 terms (n = 0, 1, 2, 3)
vdm_closed = try_summation(s, [(1,1,-3), (1,1,2)], [(1,1,5)], 1, 1, 1, 20)
if vdm_closed is not None:
    print("Closed form found!")
print("q-Vandermonde closed form:", vdm_closed)

Closed form found!
q-Vandermonde closed form: q^6 - q^9 - q^10 + q^12 + 2*q^13 - q^15 - 2*q^16 - q^17 + q^18 + 2*q^19 + O(q^20)


The closed form $a^n \cdot (c/a;q)_n / (c;q)_n$ with $a = q^2$, $n = 3$, $c = q^5$ gives
$(q^2)^3 \cdot (q^3;q)_3 / (q^5;q)_3$, which is the polynomial above.

Note: Since terminating series involve $q^{-n}$ parameters (negative $q$-powers),
the formal power series evaluation via `phi()` does not naturally terminate.
For these identities, `try_summation` computes the exact closed form directly
from the formula.

## $q$-Pfaff-Saalschutz Formula

The **$q$-Pfaff-Saalschutz formula** (DLMF 17.7.4) gives a closed form for
balanced, terminating ${}_3\phi_2$ series:

$${}_3\phi_2\left(\begin{matrix} a, b, q^{-n} \\ c, abq^{1-n}/c \end{matrix}; q, q\right) = \frac{(c/a;q)_n\,(c/b;q)_n}{(c;q)_n\,(c/(ab);q)_n}$$

The **balance condition** requires the second lower parameter to be $d = abq^{1-n}/c$.

In [ ]:
# q-Saalschutz: 3phi2(q^{-2}, q^2, q^3; q^6, q^{-2}; q, q)
# n=2, a=q^2, b=q^3, c=q^6
# Balance: d = a*b*q^{1-n}/c = q^2*q^3*q^{-1}/q^6 = q^{-2}
saal_closed = try_summation(s, [(1,1,-2), (1,1,2), (1,1,3)], [(1,1,6), (1,1,-2)], 1, 1, 1, 20)
if saal_closed is not None:
    print("Saalschutz closed form found!")
print("Closed:", saal_closed)

Saalschutz closed form found!
Closed: 1 + q + 2*q^2 + q^3 - 2*q^5 - 2*q^6 - q^7 + q^8 + 3*q^9 + 2*q^10 - 3*q^12 - 3*q^13 - q^14 + 2*q^15 + 4*q^16 + 2*q^17 - q^18 - 4*q^19 + O(q^20)


In [ ]:
# Verify: direct phi() evaluation matches the closed form
saal_direct = phi(s, [(1,1,-2), (1,1,2), (1,1,3)], [(1,1,6), (1,1,-2)], 1, 1, 1, 20)
print("Direct: ", saal_direct)
print("Match:", repr(saal_direct) == repr(saal_closed))

Direct:  1 + q + 2*q^2 + q^3 - 2*q^5 - 2*q^6 - q^7 + q^8 + 3*q^9 + 2*q^10 - 3*q^12 - 3*q^13 - q^14 + 2*q^15 + 4*q^16 + 2*q^17 - q^18 - 4*q^19 + O(q^20)
Match: True


The direct evaluation and the Saalschutz closed form agree perfectly, confirming
the identity.

## $q$-Kummer (Bailey-Daum) Summation

The **$q$-Kummer formula** (DLMF 17.6.5, also called the Bailey-Daum summation)
evaluates a ${}_2\phi_1$ at a specific argument involving $q^2$-Pochhammer symbols:

$${}_2\phi_1\left(\begin{matrix} a, b \\ aq/b \end{matrix}; q, -q/b\right) = \frac{(-q;q)_\infty\,(aq;q^2)_\infty\,(aq^2/b^2;q^2)_\infty}{(-q/b;q)_\infty\,(aq/b;q)_\infty}$$

The conditions are: lower parameter $c = aq/b$ and argument $z = -q/b$.
Note the $q^2$-Pochhammer symbols $(\cdot\,;q^2)_\infty$ in the closed form,
which make this formula structurally different from the others.

In [ ]:
# q-Kummer: 2phi1(q^3, q; q^3; q, -1)
# a=q^3, b=q: c = aq/b = q^3*q/q = q^3, z = -q/b = -q/q = -1
# The argument z = -1 is specified as (-1, 1, 0) meaning -1 * q^0
kummer_closed = try_summation(s, [(1,1,3), (1,1,1)], [(1,1,3)], -1, 1, 0, 20)
if kummer_closed is not None:
    print("Kummer closed form found!")
print("Closed:", kummer_closed)

Kummer closed form found!
Closed: 1/2 + O(q^20)


The closed form evaluates to $1/2$ (a constant). This is because with $a = q^3$
and $b = q$, the infinite product ratio in the Kummer formula simplifies to
$(-q;q)_\infty \cdot (q^4;q^2)_\infty \cdot (q^4;q^2)_\infty /
[(-1;q)_\infty \cdot (q^3;q)_\infty]$, which happens to be $1/2$.

Note: Since the argument $z = -1$ involves a non-$q$-power coefficient,
the direct `phi()` evaluation produces a different formal power series.
The `try_summation` function computes the true closed form from the
infinite product formula.

## $q$-Dixon (Jackson) Summation

The **$q$-Dixon formula** (DLMF 17.7.6) evaluates a special terminating
${}_3\phi_2$:

$${}_3\phi_2\left(\begin{matrix} q^{-2n}, b, c \\ q^{1-2n}/b,\; q^{1-2n}/c \end{matrix}; q, \frac{q^{2-n}}{bc}\right) = \frac{(b;q)_n\,(c;q)_n\,(q;q)_{2n}\,(bc;q)_{2n}}{(q;q)_n\,(bc;q)_n\,(b;q)_{2n}\,(c;q)_{2n}}$$

The key structural feature: the terminating parameter is $q^{-2n}$ (even exponent),
and the lower parameters depend on $b$, $c$, and $n$.

In [ ]:
# q-Dixon with n=1 (m=2n=2), b=q, c=q^2
# upper: [q^{-2}, q, q^2]
# lower: [q^{1-2}/q, q^{1-2}/q^2] = [q^{-2}, q^{-3}]
# z: q^{2-1}/(q*q^2) = q/(q^3) = q^{-2}
dixon1 = try_summation(s, [(1,1,-2), (1,1,1), (1,1,2)], [(1,1,-2), (1,1,-3)], 1, 1, -2, 20)
print("Dixon n=1:", dixon1)

# q-Dixon with n=2 (m=2n=4), b=q, c=q^2
# upper: [q^{-4}, q, q^2]
# lower: [q^{-3}/q, q^{-3}/q^2] = [q^{-4}, q^{-5}]
# z: q^{2-2}/(q*q^2) = 1/q^3 = q^{-3}
dixon2 = try_summation(s, [(1,1,-4), (1,1,1), (1,1,2)], [(1,1,-4), (1,1,-5)], 1, 1, -3, 20)
print("Dixon n=2:", dixon2)

Dixon n=1: 1 + q^3 - q^4 + q^6 - q^7 + q^9 - q^10 + q^12 - q^13 + q^15 - q^16 + q^18 - q^19 + O(q^20)
Dixon n=2: 1 + q^4 - q^6 + q^8 - q^10 + q^12 - q^14 + q^16 - q^18 + O(q^20)


The Dixon formula produces closed forms involving ratios of finite
$q$-Pochhammer products. Notice the alternating-sign pattern in the coefficients.

## Heine's Second Transformation

**Heine's second transformation** (DLMF 17.6.2) provides another rewriting of
a ${}_2\phi_1$:

$${}_2\phi_1(a, b; c; q, z) = \frac{(c/b;q)_\infty\,(bz;q)_\infty}{(c;q)_\infty\,(z;q)_\infty} \cdot {}_2\phi_1\!\left(\frac{abz}{c}, b; bz; q, \frac{c}{b}\right)$$

We demonstrate on ${}_2\phi_1(q^2, q^3; q^6; q, q^2)$ to compare all three
Heine transformations:

In [ ]:
from q_kangaroo import heine2

# Direct evaluation of 2phi1(q^2, q^3; q^6; q, q^2)
direct_h = phi(s, [(1,1,2), (1,1,3)], [(1,1,6)], 1, 1, 2, 20)
print("Direct:    ", direct_h)

# Apply Heine's second transformation
prefactor2, result2 = heine2(s, [(1,1,2), (1,1,3)], [(1,1,6)], 1, 1, 2, 20)
print("Heine 2:   ", result2)
print("Match:", repr(direct_h) == repr(result2))

Direct:     1 + q^2 + q^3 + q^4 + q^6 + q^8 + q^12 + q^18 + O(q^20)
Heine 2:    1 + q^2 + q^3 + q^4 + q^6 + q^8 + q^12 + q^18 + O(q^20)
Match: True


## Heine's Third Transformation

**Heine's third transformation** (DLMF 17.6.3) is the composition of the first
and second transformations:

$${}_2\phi_1(a, b; c; q, z) = \frac{(abz/c;q)_\infty}{(z;q)_\infty} \cdot {}_2\phi_1\!\left(\frac{c}{a}, \frac{c}{b}; c; q, \frac{abz}{c}\right)$$

Note the simpler prefactor (a single ratio of infinite products) compared to
the first two transformations.

In [ ]:
from q_kangaroo import heine3

# Apply Heine's third transformation
prefactor3, result3 = heine3(s, [(1,1,2), (1,1,3)], [(1,1,6)], 1, 1, 2, 20)
print("Heine 3:   ", result3)
print("Match:", repr(direct_h) == repr(result3))

Heine 3:    1 + q^2 + q^3 + q^4 + q^6 + q^8 + q^12 + q^18 + O(q^20)
Match: True


## Comparing All Three Heine Transformations

A key property: all three Heine transformations applied to the same
${}_2\phi_1$ must produce the same final series (the prefactor times the
transformed series equals the original in each case).

In [ ]:
# Apply all three Heine transformations to 2phi1(q^2, q^3; q^6; q, q^2)
p1, r1 = heine1(s, [(1,1,2), (1,1,3)], [(1,1,6)], 1, 1, 2, 20)
p2, r2 = heine2(s, [(1,1,2), (1,1,3)], [(1,1,6)], 1, 1, 2, 20)
p3, r3 = heine3(s, [(1,1,2), (1,1,3)], [(1,1,6)], 1, 1, 2, 20)
print("All match:", repr(r1) == repr(r2) == repr(r3))

print("\nPrefactors (different for each transformation):")
print("  Heine 1:", p1)
print("  Heine 2:", p2)
print("  Heine 3:", p3)

All match: True

Prefactors (different for each transformation):
  Heine 1: 1 + q^2 - q^5 - q^7 + O(q^20)
  Heine 2: 1 + q^2 + q^4 - q^5 + q^6 - q^7 + q^8 - q^9 + q^10 - q^11 + q^12 - q^13 + q^14 - q^15 + q^16 - q^17 + q^18 - q^19 + O(q^20)
  Heine 3: 1 - q + O(q^20)


While the final series value is the same, each transformation produces a
different prefactor and a different transformed ${}_2\phi_1$. The product
prefactor $\times$ transformed series is invariant.

## Bilateral Basic Hypergeometric Series

The **bilateral** series ${}_r\psi_s$ sums from $n = -\infty$ to $+\infty$
instead of from $n = 0$ to $+\infty$:

$${}_r\psi_s\left(\begin{matrix} a_1, \ldots, a_r \\ b_1, \ldots, b_s \end{matrix}; q, z\right) = \sum_{n=-\infty}^{\infty} \frac{(a_1;q)_n \cdots (a_r;q)_n}{(b_1;q)_n \cdots (b_s;q)_n} \left[(-1)^n q^{\binom{n}{2}}\right]^{s-r} z^n$$

Note: no $(q;q)_n$ in the denominator (unlike $\phi$), and the extra factor
exponent is $s - r$ (not $1 + s - r$).

The `psi` function evaluates bilateral series, including contributions from
negative indices:

In [ ]:
from q_kangaroo import psi

# Bilateral 1psi1(q^2; q^5; q, q)
bilateral = psi(s, [(1,1,2)], [(1,1,5)], 1, 1, 1, 20)
print("1psi1(q^2; q^5; q, q) =", bilateral)

1psi1(q^2; q^5; q, q) = q^-1 + 2 + 2*q + 2*q^2 - q^5 + O(q^20)


Notice the **negative power** $q^{-1}$ in the output -- this comes from the
$n = -1$ term in the bilateral sum. Bilateral series can have terms with
negative $q$-powers, which distinguishes them from the unilateral $\phi$ series.

The famous **Ramanujan's ${}_1\psi_1$ summation** gives a closed form for
the bilateral ${}_1\psi_1(a; b; q, z)$ as a ratio of infinite products.
Bilateral series are closely connected to theta functions and modular forms.

## Summary of Summation Formulas

| Formula | Series Type | Key Condition | DLMF |
|---------|------------|---------------|------|
| $q$-Gauss | ${}_2\phi_1$ | $z = c/(ab)$ | 17.7.2 |
| $q$-Vandermonde | ${}_2\phi_1$ | $q^{-n}$ upper, $z = q$ or $z = cq^n/a$ | 17.6.2 |
| $q$-Saalschutz | ${}_3\phi_2$ | $q^{-n}$ upper, balanced, $z = q$ | 17.7.4 |
| $q$-Kummer | ${}_2\phi_1$ | $c = aq/b$, $z = -q/b$ | 17.6.5 |
| $q$-Dixon | ${}_3\phi_2$ | $q^{-2n}$ upper, specific structure | 17.7.6 |

The `try_summation` function automatically checks all five formulas and returns
the closed form if any pattern matches.

## References

- DLMF Chapter 17: [$q$-Hypergeometric and Related Functions](https://dlmf.nist.gov/17)
- DLMF 17.6: Transformations of ${}_r\phi_s$ Functions (Heine, Kummer)
- DLMF 17.7: Special Cases of ${}_r\phi_s$ Functions (Gauss, Vandermonde, Saalschutz, Dixon)
- G. Gasper and M. Rahman, *Basic Hypergeometric Series*, 2nd ed., Cambridge University Press, 2004
- W.N. Bailey, *Generalized Hypergeometric Series*, Cambridge University Press, 1935